![picture](https://drive.google.com/uc?export=view&id=1K_GhwMAZaYPEqm-0ukMBxfmBew7N6GXA)<br>
<small>Rian Lopes</small><br>

Inscreva-se no canal <br>[Data Review](https://www.youtube.com/channel/UCYfY8KRS5nqoFBTuoLV0_jw) <br>
E siga nas redes sociais <br> [Instagram Data Review](https://www.instagram.com/data.review/)
<br>
<br>
<h1>Spark for Data Science and Machine Learning</h1>
<h2>Data Strucures Interfaces</h2>


No Spark existem três principais interfaces para manipulação dos dados de maneira distribuída:



*   RDD - Resilient Distribuited Dataset
*   DataFrame
*   Dataset*


*No PySpark apenas duas são realmente usadas, pois Dataset é útil apenas em Scala/Java.

In [ ]:
# Inicialização do Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("EstruturasSpark") \
    .getOrCreate()

<h2>RDD</h2>
RDD  ou Resilient Distributed Dataset, é a estrutura de mais baixo nível e fundamental no Spark.
Estrutura imutável, tolerante a falhas e distribuída.
Usa transformações como map(), filter(), reduce().<br>
Menos otimizado, pois não tem um schema explícito.<br>

📌 Quando usar?
Quando precisar de processamento mais flexível e baixo nível.
Quando os dados não têm um schema fixo (dados semi-estruturados).
Quando quiser controle total sobre a distribuição dos dados.

In [ ]:
# Criando um RDD a partir de uma lista
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
rdd = spark.sparkContext.parallelize(data)

# # Exibindo o conteúdo do RDD
print(rdd.collect())

[('Alice', 25), ('Bob', 30), ('Charlie', 35)]


In [ ]:
print(rdd.count())

3


In [ ]:
rdd1 = spark.sparkContext.emptyRDD()
rdd1 = rdd1.union(spark.sparkContext.parallelize([("John", 25), ("Mary", 30), ("Carl", 35)]))
print(rdd1.collect())

[('John', 25), ('Mary', 30), ('Carl', 35)]


In [ ]:
rdd_union = rdd.union(rdd1)
print(rdd_union.collect())

[('Alice', 25), ('Bob', 30), ('Charlie', 35), ('John', 25), ('Mary', 30), ('Carl', 35)]


In [ ]:
rdd_union_map = rdd_union.map(lambda x: (x[0], x[1] + 5))
print(rdd_union_map.collect())

[('Alice', 30), ('Bob', 35), ('Charlie', 40), ('John', 30), ('Mary', 35), ('Carl', 40)]


In [ ]:
rdd_filter = rdd_union_map.filter(lambda x: x[1] > 30)
print(rdd_filter.collect())

[('Bob', 35), ('Charlie', 40), ('Mary', 35), ('Carl', 40)]


In [ ]:
rdd_sorted = rdd_filter.sortBy(lambda x: x[1], ascending=False)
print(rdd_sorted.collect())

[('Charlie', 40), ('Carl', 40), ('Bob', 35), ('Mary', 35)]


In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

# Carregando o dataset Iris
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df["target"] = iris.target  # Adicionando a coluna de classes

# Salvando como CSV
iris_df.to_csv("iris.csv", index=False)


In [ ]:
# Criando um RDD a partir do arquivo
rdd_iris = spark.sparkContext.textFile("iris.csv")

# Exibindo as 5 primeiras linhas do RDD
print(rdd_iris.take(5))


['sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target', '5.1,3.5,1.4,0.2,0', '4.9,3.0,1.4,0.2,0', '4.7,3.2,1.3,0.2,0', '4.6,3.1,1.5,0.2,0']


<h2>DataFrame</h2>

Estrutura mais usada no PySpark (equivalente ao Pandas DataFrame).
Otimizado pelo Catalyst Optimizer para performance.
Suporte a SQL e manipulação via select(), groupBy(), join().
Possui schema explícito, facilitando manipulação de dados estruturados.<br><br>
📌 Quando usar?

Sempre que os dados forem estruturados (CSV, JSON, Parquet, banco de dados).
Quando precisar de consultas SQL e operações otimizadas.
Quando quiser melhor desempenho em grandes volumes de dados.

In [ ]:
from pyspark.sql import Row

# Criando um DataFrame a partir de uma lista de tuplas
data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)

# Exibindo o DataFrame
df.show()


+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [ ]:
rdd_from_df = df.rdd
print(rdd_from_df.collect())

[Row(Name='Alice', Age=25), Row(Name='Bob', Age=30), Row(Name='Charlie', Age=35)]


In [ ]:
# Criando um arquivo CSV de exemplo

# Lendo um arquivo CSV para um DataFrame do Spark
df_csv = spark.read.csv("iris.csv", header=True, inferSchema=True)

# Exibindo o DataFrame lido
df_csv.show()


+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              5.1|             3.5|              1.4|             0.2|     0|
|              4.9|             3.0|              1.4|             0.2|     0|
|              4.7|             3.2|              1.3|             0.2|     0|
|              4.6|             3.1|              1.5|             0.2|     0|
|              5.0|             3.6|              1.4|             0.2|     0|
|              5.4|             3.9|              1.7|             0.4|     0|
|              4.6|             3.4|              1.4|             0.3|     0|
|              5.0|             3.4|              1.5|             0.2|     0|
|              4.4|             2.9|              1.4|             0.2|     0|
|              4.9|             3.1|              1.

In [ ]:
df_csv.printSchema()

root
 |-- sepal length (cm): double (nullable = true)
 |-- sepal width (cm): double (nullable = true)
 |-- petal length (cm): double (nullable = true)
 |-- petal width (cm): double (nullable = true)
 |-- target: integer (nullable = true)



In [ ]:
df_pandas

,Name,Age
0,Alice,25
1,Bob,30
2,Charlie,35


In [ ]:
import pandas as pd

# Criando um DataFrame do Pandas
df_pandas = pd.DataFrame(data, columns=columns)

# Convertendo para um DataFrame do Spark
df_spark = spark.createDataFrame(df_pandas)

# Exibindo o DataFrame
df_spark.show()


+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [ ]:
df_spark.toPandas()

,Name,Age
0,Alice,25
1,Bob,30
2,Charlie,35


\begin{array}{|l|l|l|}
\hline
\textbf{Característica} & \textbf{RDD} & \textbf{DataFrame} \\ \hline
\textbf{Estrutura} & Sem\quad schema & Schema\quad estruturado \\ \hline
\textbf{Performance} & Menos\quad otimizado & Melhor\quad otimização \\ \hline
\textbf{Suporte a SQL} & Não & Sim \\ \hline
\textbf{Transformações} & map, filter, reduce & select, groupBy, join \\ \hline
\end{array}

